## Leo CSV de películas

In [28]:
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('movies/movie_titles_1000.csv')
dataframe = pycsv.csvToDataFrame(sqlCtx, plaintext_rdd, parseDate=False, columns=['uri','attribute','movie'])
data_movies = dataframe.rdd
data_movies.first()

Row(uri=u'<http://data.linkedmdb.org/resource/film/1>', attribute=u'<http://purl.org/dc/terms/title>', movie=u'Buffy the Vampire Slayer')

## Leo CSV de actores

In [29]:
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('movies/actor_names_10000.csv')
dataframe = pycsv.csvToDataFrame(sqlCtx, plaintext_rdd, parseDate=False, columns=['uri','attribute','actor'])
data_actors = dataframe.rdd
data_actors.first()

Row(uri=u'<http://data.linkedmdb.org/resource/actor/1>', attribute=u'<http://data.linkedmdb.org/resource/movie/actor_name>', actor=u'Bernard Montgomery, 1st Viscount Montgomery of Alamein')

## Leo CSV de actuaciones en películas

In [30]:
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('movies/actor_movies_m.csv')
dataframe = pycsv.csvToDataFrame(sqlCtx, plaintext_rdd, parseDate=False, columns=['movie_uri','attribute','actor_uri'])
data_actor_movies = dataframe.rdd
data_actor_movies.first()

Row(movie_uri=u'<http://data.linkedmdb.org/resource/film/54176>', attribute=u'<http://data.linkedmdb.org/resource/movie/actor>', actor_uri=u'<http://data.linkedmdb.org/resource/actor/29362>')

## Genero un RDD con el id de la película como key

In [31]:
movies = data_movies.map(lambda row: (row.uri, row.movie))
movies.first()

(u'<http://data.linkedmdb.org/resource/film/1>', u'Buffy the Vampire Slayer')

## Genero un RDD de las actuaciones con el id de la película como key

In [32]:
actor_movies_by_movie = data_actor_movies.map(lambda row: (row.movie_uri, row.actor_uri))
actor_movies_by_movie.first()

(u'<http://data.linkedmdb.org/resource/film/54176>',
 u'<http://data.linkedmdb.org/resource/actor/29362>')

## Join de los dos RDD anteriores para tener para cada película el nombre y el actor

In [33]:
movies_join_actors = movies.join(actor_movies_by_movie)
movies_join_actors.first()

(u'<http://data.linkedmdb.org/resource/film/673>',
 (u"The Devil's Disciple",
  u'<http://data.linkedmdb.org/resource/actor/29396>'))

## Genero un RDD con el id del actor como key

In [34]:
actors = data_actors.map(lambda row: (row.uri, row.actor))
actors.first()

(u'<http://data.linkedmdb.org/resource/actor/1>',
 u'Bernard Montgomery, 1st Viscount Montgomery of Alamein')

## Actuaciones con id de actor como key

In [35]:
actor_movies_by_actor = data_actor_movies.map(lambda row: (row.actor_uri, row.movie_uri))
actor_movies_by_actor.first()

(u'<http://data.linkedmdb.org/resource/actor/29362>',
 u'<http://data.linkedmdb.org/resource/film/54176>')

## Actuaciones con id de actor como key y con el nombre del actor

In [37]:
actor_movies_with_actor = actor_movies_by_actor.join(actors)
actor_movies_with_actor.first()

(u'<http://data.linkedmdb.org/resource/actor/7734>',
 (u'<http://data.linkedmdb.org/resource/film/48023>', u'Tom Choi'))

## El anterior pero con el id de película como key

In [38]:
actor_movies_with_actor_by_movie = actor_movies_with_actor.map(lambda row: (row[1][0], (row[0],row[1][1])))
actor_movies_with_actor_by_movie.first()

(u'<http://data.linkedmdb.org/resource/film/48023>',
 (u'<http://data.linkedmdb.org/resource/actor/7734>', u'Tom Choi'))

## El anterior agregado el nombre de la película

In [39]:
actor_movies_with_actor_by_movie_with_movie = actor_movies_with_actor_by_movie.join(movies)
actor_movies_with_actor_by_movie_with_movie.first()

(u'<http://data.linkedmdb.org/resource/film/1031>',
 ((u'<http://data.linkedmdb.org/resource/actor/420>', u'Andrew Wilson'),
  u'Bottle Rocket'))

## Junto todos los actores de una misma película

In [45]:
actor_movies_with_actor_by_movie_with_movie.groupByKey().mapValues(lambda x: list(x)).take(5)

[(u'<http://data.linkedmdb.org/resource/film/512>',
  [((u'<http://data.linkedmdb.org/resource/actor/872>', u'Walter Long'),
    u'Intolerance')]),
 (u'<http://data.linkedmdb.org/resource/film/822>',
  [((u'<http://data.linkedmdb.org/resource/actor/9758>', u'Suhasini Mulay'),
    u'Lagaan: Once Upon a Time in India'),
   ((u'<http://data.linkedmdb.org/resource/actor/9515>',
     u'Rajendranath Zutshi'),
    u'Lagaan: Once Upon a Time in India')]),
 (u'<http://data.linkedmdb.org/resource/film/176>',
  [((u'<http://data.linkedmdb.org/resource/actor/7463>', u'J. Frank Burke'),
    u'The Italian'),
   ((u'<http://data.linkedmdb.org/resource/actor/7462>', u'George Beban'),
    u'The Italian')]),
 (u'<http://data.linkedmdb.org/resource/film/839>',
  [((u'<http://data.linkedmdb.org/resource/actor/7594>', u'Michael Scholes'),
    u'The Lord of the Rings'),
   ((u'<http://data.linkedmdb.org/resource/actor/7598>', u'David Buck'),
    u'The Lord of the Rings'),
   ((u'<http://data.linkedmdb.org/r

## Uso de cogroup para hacer join y agrupar

In [74]:
actor_movies_by_actor.cogroup(actors)\
                     .mapValues(lambda x: (list(x[0]), list(x[1])))\
                     .filter(lambda x: len(x[1][1]) > 0)\
                     .filter(lambda x: len(x[1][0]) > 0)\
                     .takeSample(0, 5)

[(u'<http://data.linkedmdb.org/resource/actor/349>',
  ([u'<http://data.linkedmdb.org/resource/film/4706>'],
   [u'Robert Stevenson'])),
 (u'<http://data.linkedmdb.org/resource/actor/465>',
  ([u'<http://data.linkedmdb.org/resource/film/43596>',
    u'<http://data.linkedmdb.org/resource/film/11098>'],
   [u'Doug Stone'])),
 (u'<http://data.linkedmdb.org/resource/actor/9609>',
  ([u'<http://data.linkedmdb.org/resource/film/2436>',
    u'<http://data.linkedmdb.org/resource/film/5682>',
    u'<http://data.linkedmdb.org/resource/film/46765>',
    u'<http://data.linkedmdb.org/resource/film/8346>',
    u'<http://data.linkedmdb.org/resource/film/8775>'],
   [u'Barbara Jefford'])),
 (u'<http://data.linkedmdb.org/resource/actor/2227>',
  ([u'<http://data.linkedmdb.org/resource/film/3582>'],
   [u'Terri Susan Smith'])),
 (u'<http://data.linkedmdb.org/resource/actor/612>',
  ([u'<http://data.linkedmdb.org/resource/film/42279>'], [u'James W. Horne']))]